In [18]:
import pandas as pd
import yfinance as yf
import numpy as np
from sqlalchemy import create_engine
from ta import add_all_ta_features
import plotly.graph_objs as go
import pandas_ta as ta

In [2]:
df = yf.download("SPY", period = "60d", interval = "5m")

[*********************100%%**********************]  1 of 1 completed


In [15]:
df["Date"] = pd.to_datetime(df.index ,format='%d.%m.%Y %H:%M:%S')

df=df[df.High!=df.Low]
df.set_index("Date", inplace=True)

In [19]:
df["EMA_slow"]=ta.ema(df.Close, length=50)
df["EMA_fast"]=ta.ema(df.Close, length=30)
df['RSI']=ta.rsi(df.Close, length=10)
my_bbands = ta.bbands(df.Close, length=15, std=1.5)
df['ATR']=ta.atr(df.High, df.Low, df.Close, length=7)
df=df.join(my_bbands)
df

,Open,High,Low,Close,Adj Close,Volume,EMA_slow,EMA_fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5
Date,,,,,,,,,,,,,,,
2023-11-06 09:30:00-05:00,435.470001,436.149811,435.459991,435.750000,435.750000,2651766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-06 09:35:00-05:00,435.739990,435.945007,435.450012,435.808899,435.808899,908117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-06 09:40:00-05:00,435.804993,435.839996,435.130005,435.380005,435.380005,1155551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-06 09:45:00-05:00,435.380005,435.480011,435.010010,435.239990,435.239990,836980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-06 09:50:00-05:00,435.250000,435.424988,435.065002,435.279694,435.279694,700273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-01 15:35:00-05:00,488.779999,488.920013,488.670013,488.910004,488.910004,827126,487.407085,487.872329,68.839784,0.333481,487.504961,488.318335,489.131709,0.333133,0.863712
2024-02-01 15:40:00-05:00,488.899994,489.079987,488.755005,488.760010,488.760010,1154267,487.460141,487.929599,63.565585,0.332267,487.659546,488.406335,489.153125,0.305807,0.736796
2024-02-01 15:45:00-05:00,488.750000,488.899994,488.350006,488.359985,488.359985,1724882,487.495429,487.957366,51.804333,0.363370,487.950861,488.489669,489.028476,0.220601,0.379657


In [20]:
def ema_signal(df, current_candle, backcandles):
    df_slice = df.reset_index().copy()
    # Get the range of candles to consider
    start = max(0, current_candle - backcandles)
    end = current_candle
    relevant_rows = df_slice.iloc[start:end]

    # Check if all EMA_fast values are below EMA_slow values
    if all(relevant_rows["EMA_fast"] < relevant_rows["EMA_slow"]):
        return 1
    elif all(relevant_rows["EMA_fast"] > relevant_rows["EMA_slow"]):
        return 2
    else:
        return 0


df=df[-10000:-1]
from tqdm import tqdm
tqdm.pandas()
df.reset_index(inplace=True)
df['EMASignal'] = df.progress_apply(lambda row: ema_signal(df, row.name, 7) , axis=1) #if row.name >= 20 else 0

100%|██████████| 4641/4641 [00:06<00:00, 675.65it/s]
C:\Users\Vishnu\AppData\Local\Temp\ipykernel_12636\3663350097.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EMASignal'] = df.progress_apply(lambda row: ema_signal(df, row.name, 7) , axis=1) #if row.name >= 20 else 0


In [21]:
def total_signal(df, current_candle, backcandles):
    if (ema_signal(df, current_candle, backcandles)==2
        and df.Close[current_candle]<=df['BBL_15_1.5'][current_candle]
        #and df.RSI[current_candle]<60
        ):
            return 2
    if (ema_signal(df, current_candle, backcandles)==1
        and df.Close[current_candle]>=df['BBU_15_1.5'][current_candle]
        #and df.RSI[current_candle]>40
        ):
    
            return 1
    return 0
        
df['TotalSignal'] = df.progress_apply(lambda row: total_signal(df, row.name, 7), axis=1)

100%|██████████| 4641/4641 [00:14<00:00, 311.28it/s]
C:\Users\Vishnu\AppData\Local\Temp\ipykernel_12636\3699091765.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalSignal'] = df.progress_apply(lambda row: total_signal(df, row.name, 7), axis=1)


In [22]:
df[df.TotalSignal != 0].head(20)

,Date,Open,High,Low,Close,Adj Close,Volume,EMA_slow,EMA_fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5,EMASignal,TotalSignal
60,2023-11-06 14:30:00-05:00,434.279999,434.570007,434.190002,434.535004,434.535004,736789,434.791336,434.481889,56.477946,0.340845,433.898185,434.194843,434.491500,0.136647,1.073323,1,1
62,2023-11-06 14:40:00-05:00,434.529999,435.040588,434.440002,434.920013,434.920013,807210,434.786535,434.513059,65.458686,0.365607,433.846668,434.262844,434.679020,0.191670,1.289534,1,1
63,2023-11-06 14:45:00-05:00,434.910004,435.029999,434.765015,434.790009,434.790009,1240531,434.786671,434.530927,60.745533,0.351231,433.845645,434.303845,434.762045,0.211004,1.030514,1,1
64,2023-11-06 14:50:00-05:00,434.790009,435.119995,434.774994,435.070007,435.070007,600011,434.797783,434.565706,66.515145,0.350341,433.804937,434.343512,434.882087,0.247995,1.174460,1,1
65,2023-11-06 14:55:00-05:00,435.059998,435.170013,434.994995,435.089996,435.089996,509328,434.809242,434.599531,66.901036,0.325294,433.782564,434.389532,434.996501,0.279458,1.077018,1,1
66,2023-11-06 15:00:00-05:00,435.089996,435.140015,434.959991,435.129791,435.129791,535714,434.821812,434.633742,67.723832,0.304540,433.788605,434.450326,435.112046,0.304624,1.013409,1,1
77,2023-11-06 15:55:00-05:00,435.040009,435.739990,435.019989,435.649994,435.649994,5962917,434.911678,434.866395,74.524925,0.338071,434.777195,435.052891,435.328587,0.126741,1.582900,1,1
83,2023-11-07 09:55:00-05:00,435.350006,435.600006,435.329987,435.554993,435.554993,949119,434.949196,434.943212,61.942427,0.426689,434.696727,435.073905,435.451084,0.173386,1.137744,1,1
84,2023-11-07 10:00:00-05:00,435.549988,436.049988,435.540009,435.920013,435.920013,1161034,434.987267,435.006232,67.317810,0.438588,434.636573,435.129240,435.621907,0.226446,1.302544,1,1
108,2023-11-07 12:00:00-05:00,436.885010,436.889893,436.429993,436.450012,436.450012,574437,436.124168,436.488328,40.441026,0.342012,436.648545,437.005202,437.361859,0.163228,-0.278325,2,2


In [23]:
import numpy as np
def pointpos(x):
    if x['TotalSignal']==2:
        return x['Low']-1e-3
    elif x['TotalSignal']==1:
        return x['High']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

C:\Users\Vishnu\AppData\Local\Temp\ipykernel_12636\2345118794.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)


In [24]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
st=100
dfpl = df[st:st+350]
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),

                go.Scatter(x=dfpl.index, y=dfpl['BBL_15_1.5'], 
                           line=dict(color='green', width=1), 
                           name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_15_1.5'], 
                           line=dict(color='green', width=1), 
                           name="BBU"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_fast'], 
                           line=dict(color='black', width=1), 
                           name="EMA_fast"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_slow'], 
                           line=dict(color='blue', width=1), 
                           name="EMA_slow")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="entry")

fig.show()

In [25]:
def SIGNAL():
    return df.TotalSignal

In [31]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    mysize = 1
    slcoef = 1.1
    TPSLRatio = 1.5
    rsi_length = 16
    
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        #df['RSI']=ta.rsi(df.Close, length=self.rsi_length)

    def next(self):
        super().next()
        slatr = self.slcoef*self.data.ATR[-1]
        TPSLRatio = self.TPSLRatio

        # if len(self.trades)>0:
        #     if self.trades[-1].is_long and self.data.RSI[-1]>=90:
        #         self.trades[-1].close()
        #     elif self.trades[-1].is_short and self.data.RSI[-1]<=10:
        #         self.trades[-1].close()
        
        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(df, MyStrat, cash=25000, margin=1/30)

C:\Users\Vishnu\AppData\Local\Temp\ipykernel_12636\4151719609.py:36: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



In [32]:
bt.run()

Start                                     0.0
End                                    4640.0
Duration                               4640.0
Exposure Time [%]                         0.0
Equity Final [$]                      25000.0
Equity Peak [$]                       25000.0
Return [%]                                0.0
Buy & Hold Return [%]               12.090648
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                  0.0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    